# **9. 2自由度系の自由振動**

In [3]:
# --- 【Colab用設定】 ---
from matplotlib import rc
rc('animation', html='jshtml', embed_limit=50)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

# --- (A) パラメータ設定 ---
m1 = 1.0; m2 = 1.0
k1 = 1.0; k2 = 1.0

# --- (B) 固有値解析 ---
M = np.array([[m1, 0], [0, m2]])
K = np.array([[k1 + k2, -k2], [-k2, k2]])
evals, evecs = np.linalg.eigh(np.linalg.inv(M) @ K)
omega = np.sqrt(evals)

# モードベクトル（基準化）
phi1 = evecs[:, 0] / evecs[1, 0]
phi2 = evecs[:, 1] / evecs[1, 1]

# --- (C) 初期条件の設定 ---
# ここを変えて好きな揺れ方で保存してください
# 例: [1.0, 0.5] (混合), [0.618, 1.0] (1次), [-1.0, 0.618] (2次)
u0_input = np.array([1.0, 0.618])

# --- (D) モード分解 ---
Phi = np.column_stack((phi1, phi2))
q0 = np.linalg.inv(Phi) @ u0_input

# --- (E) 時刻歴応答の計算 ---
dt = 0.05
t = np.arange(0, 20, dt)

q1_t = q0[0] * np.cos(omega[0] * t)
q2_t = q0[1] * np.cos(omega[1] * t)

u_mode1 = np.outer(phi1, q1_t)
u_mode2 = np.outer(phi2, q2_t)
u_total = u_mode1 + u_mode2

# --- (F) アニメーション描画 ---
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
# 英語タイトル（エラー回避のため）
titles = ["Mode 1 Component", "Mode 2 Component", "Actual Response (Total)"]
data_list = [u_mode1, u_mode2, u_total]
lines = []
masses = []

h_floor = 2.0
radius = 0.4

for i, ax in enumerate(axes):
    ax.set_xlim(-2.0, 2.0)
    ax.set_ylim(0, h_floor * 2 + 1.5)
    ax.set_title(titles[i], fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])

    line, = ax.plot([], [], 'k-', lw=3)
    lines.append(line)

    m1_p = patches.Circle((0, h_floor), radius=radius, fc='cyan', ec='k', zorder=10)
    ax.add_patch(m1_p)
    m2_p = patches.Circle((0, h_floor*2), radius=radius, fc='cyan', ec='k', zorder=10)
    ax.add_patch(m2_p)
    masses.append((m1_p, m2_p))

    ax.axhline(0, color='k', lw=5)

def update(frame):
    for i in range(3):
        u = data_list[i]
        curr_u1, curr_u2 = u[0, frame], u[1, frame]

        lines[i].set_data([0, curr_u1, curr_u2], [0, h_floor, h_floor*2])
        masses[i][0].set_center((curr_u1, h_floor))
        masses[i][1].set_center((curr_u2, h_floor*2))

        # 色分け
        if i == 0: masses[i][0].set_fc('#aaf'); masses[i][1].set_fc('#aaf')
        if i == 1: masses[i][0].set_fc('#faa'); masses[i][1].set_fc('#faa')
        if i == 2: masses[i][0].set_fc('#afa'); masses[i][1].set_fc('#afa')

    return lines + [m for group in masses for m in group]

print("アニメーション生成中...")
plt.close() # 余分なプロットを抑制

# 動画生成
#ani = animation.FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)

# --- (G) 保存とダウンロード処理 ---
#print("動画ファイル(mp4)を保存しています... (少し時間がかかります)")

# fps=20 (interval=50ms なので 1000/50 = 20fps でリアルタイム速度)
#ani.save('2dof_mode_decomposition.mp4', writer='ffmpeg', fps=20)
#print("保存完了: 2dof_mode_decomposition.mp4")

# 自動ダウンロード
#try:
#    from google.colab import files
#    files.download('2dof_mode_decomposition.mp4')
#except ImportError:
#    print("ローカル環境に保存されました。")
#
# Colab上でも確認できるように表示
ani

Output hidden; open in https://colab.research.google.com to view.